# Prep PHEME
Code below uses https://www.kaggle.com/code/eitharyassin/fake-news-detection-on-pheme-newfeatures as a base.

In [1]:
import numpy as np 
import pandas as pd 
import os
import json
import nltk # NLP  
from nltk.corpus import stopwords  
from nltk.stem import WordNetLemmatizer 
from nltk.stem import PorterStemmer 
import re  
import contractions

In [2]:
%cd PHEME_veracity\all-rnr-annotated-threads

c:\Users\willc\Documents\wills ensemble\PHEME_veracity\all-rnr-annotated-threads


In [3]:
seed = 42

np.random.seed(seed)

In [4]:
# set the folders to iterate on
folds= ['charliehebdo-all-rnr-threads','ottawashooting-all-rnr-threads',
'ebola-essien-all-rnr-threads','prince-toronto-all-rnr-threads',
'ferguson-all-rnr-threads',	'putinmissing-all-rnr-threads',
'germanwings-crash-all-rnr-threads',
'gurlitt-all-rnr-threads','sydneysiege-all-rnr-threads']


texts = []
fav_counts = []
retweet_counts = []
date = []

username = []
account_date = []
protected = []
verified = []
followers = []
followings = []
tweets_count = []

hashtag = []
url = []

events = []
y = []

In [5]:
# the annotations on PHEME have 2 boolean values, 'misinformation' value and 'true' value, we want to convert them in a single label with values
# 'true', 'false' or 'unverified'. This values can be string or numerical, depending on the 'string' parameter of the function
def convert_annotations_data(annotation, string = True):
    if 'misinformation' in annotation.keys() and 'true'in annotation.keys():
        if int(annotation['misinformation'])==0 and int(annotation['true'])==0:
            if string:
                label = "true"
            else:
                label = 2
        elif int(annotation['misinformation'])==0 and int(annotation['true'])==1 :
            if string:
                label = "true"
            else:
                label = 1
        elif int(annotation['misinformation'])==1 and int(annotation['true'])==0 :
            if string:
                label = "false"
            else:
                label = 0
        elif int(annotation['misinformation'])==1 and int(annotation['true'])==1:
            print ("OMG! They both are 1!")
            print(annotation['misinformation'])
            print(annotation['true'])
            label = None
            
    elif 'misinformation' in annotation.keys() and 'true' not in annotation.keys():
        # all instances have misinfo label but don't have true label
        if int(annotation['misinformation'])==0:
            if string:
                label = "false"
            else:
                label = 2
        elif int(annotation['misinformation'])==1:
            if string:
                label = "false"
            else:
                label = 0
                
    elif 'true' in annotation.keys() and 'misinformation' not in annotation.keys():
        print ('Has true not misinformation')
        label = None
    else:
        print('No annotations')
        label = None
           
    return label

In [6]:
def get_info(dir1, path1, f, rumour_status):
    if '_' not in dir1:
          path_target  = os.path.join(path1,dir1,'annotation.json')
          file = open(path_target)
          data = json.load(file)
          target = rumour_status
          y.append(target)
          path2 = os.path.join(path1, dir1,'source-tweets')
          for dir2 in os.listdir(path2):
            if '_' not in dir2:
              path3  = os.path.join(path2,dir2)
              file = open(path3)
              data = json.load(file)
            
              #tweet features
              text = data['text']
              tweet_date = data['created_at']
              fav = data['favorite_count']
              retw = data['retweet_count']
                
              #user features
              usernames = data['user']['screen_name']
              account_creation = data['user']['created_at']
              is_protected = data['user']['protected']
              is_verified = data['user']['verified']
              no_followers = data['user']['followers_count']
              no_followings = data['user']['friends_count']
              no_tweets = data['user']['statuses_count']
                
              #entities
              no_hashtags = len(data['entities']['hashtags'])      
              has_url = data['entities']['urls']      
   

              texts.append(text)
              date.append(tweet_date)
              fav_counts.append(fav)
              retweet_counts.append(retw)
                                     
              username.append(usernames)
              account_date.append(account_creation)
              protected.append(is_protected)
              verified.append(is_verified)
              followers.append(no_followers)
              followings.append(no_followings)
              tweets_count.append(no_tweets)
            
              
              hashtag.append(no_hashtags)
              url.append(has_url)
            
              events.append(f)

In [7]:
# all interesting features
for f in folds:
  rumpath = os.path.join(f, 'rumours')
  nonrumpath = os.path.join(f, 'non-rumours')
  
  for dir1 in os.listdir(rumpath):
    get_info(dir1, rumpath, f, "true")
  for dir1 in os.listdir(nonrumpath):
    get_info(dir1, nonrumpath, f,"false")
        
              

In [8]:
df = pd.DataFrame([texts,date,fav_counts,retweet_counts,username,account_date,followers,followings,tweets_count,protected,verified,hashtag,url,events,y],['text','date','fav_count','retweet_count','username','account_date','followers','followings','tweet_count','protected','verified','no_hashtags','urls','event','target']).transpose()
df = df.infer_objects()
df.head()

,text,date,fav_count,retweet_count,username,account_date,followers,followings,tweet_count,protected,verified,no_hashtags,urls,event,target
0,"Breaking: At least 10 dead, 5 injured after tO...",Wed Jan 07 11:06:08 +0000 2015,14,159,H_E_Samuel,Tue Oct 04 09:36:17 +0000 2011,1628,246,1901,False,False,0,[],charliehebdo-all-rnr-threads,true
1,France: 10 people dead after shooting at HQ of...,Wed Jan 07 11:07:51 +0000 2015,38,486,euronews,Wed Mar 18 12:57:11 +0000 2009,129573,337,48440,False,True,1,[],charliehebdo-all-rnr-threads,true
2,Ten killed in shooting at headquarters of Fren...,Wed Jan 07 11:08:09 +0000 2015,15,127,Channel4News,Mon Apr 28 14:29:37 +0000 2008,529882,3051,66682,False,True,1,[],charliehebdo-all-rnr-threads,true
3,BREAKING: 10 dead in shooting at headquarters ...,Wed Jan 07 11:09:50 +0000 2015,15,105,AlArabiya_Eng,Sat Feb 28 08:31:32 +0000 2009,499741,31,100466,False,True,2,[],charliehebdo-all-rnr-threads,true
4,Reuters: 10 people shot dead at headquarters o...,Wed Jan 07 11:11:16 +0000 2015,32,412,SkyNewsBreak,Wed Nov 04 11:25:45 +0000 2009,1377384,6,27228,False,True,1,[],charliehebdo-all-rnr-threads,true


In [9]:
df.shape

(6425, 15)

In [10]:
# download nltk stuff
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\willc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\willc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\willc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
# Clean text

def remove_punctuations(text):
    return re.sub('\[[^]]*\]', '', text)

def remove_characters(text):
    return re.sub('[^a-zA-Z]', ' ', text)

def remove_url(text):
    return re.sub(r'http\S+', '', text)

def remove_contractions(text):
    return ' '.join([contractions.fix(word) for word in text.split()])

stop_words = stopwords.words('english')
def remove_stopwords(text):
    return ' '.join([word for word in nltk.word_tokenize(text) if word not in stop_words])
# str(text).split()

#defining the object for Lemmatization
lemmatizer = WordNetLemmatizer()
#defining a function for lemming
def lemmatize_words(text):
    return ' '.join(lemmatizer.lemmatize(word) for word in text.split())

#defining the object for stemming
stemmer = PorterStemmer()
#defining a function for stemming
def stemming_words(text):
    return ' '.join(stemmer.stem(word) for word in text.split())

def clean_text(text):
    text = remove_url(text)
    text = remove_contractions(text)
    text = text.lower()
    text = remove_punctuations(text)
    text = remove_characters(text)
    text = remove_stopwords(text)
    text = stemming_words(text)
    text = lemmatize_words(text)
    return text

def clean_text2(text):
    text = remove_url(text)
    text = remove_contractions(text)
    #text = text.lower()
    text = remove_punctuations(text)
    #text = remove_characters(text)
    #text = remove_stopwords(text)
    return text


In [12]:
# apply
df2 = df.copy()
df['text'] = df['text'].apply(clean_text)
df2['text'] = df2['text'].apply(clean_text2)

In [13]:
df

,text,date,fav_count,retweet_count,username,account_date,followers,followings,tweet_count,protected,verified,no_hashtags,urls,event,target
0,break least dead injur gunman open fire offic ...,Wed Jan 07 11:06:08 +0000 2015,14,159,H_E_Samuel,Tue Oct 04 09:36:17 +0000 2011,1628,246,1901,False,False,0,[],charliehebdo-all-rnr-threads,true
1,franc peopl dead shoot hq satir weekli newspap...,Wed Jan 07 11:07:51 +0000 2015,38,486,euronews,Wed Mar 18 12:57:11 +0000 2009,129573,337,48440,False,True,1,[],charliehebdo-all-rnr-threads,true
2,ten kill shoot headquart french satir weekli c...,Wed Jan 07 11:08:09 +0000 2015,15,127,Channel4News,Mon Apr 28 14:29:37 +0000 2008,529882,3051,66682,False,True,1,[],charliehebdo-all-rnr-threads,true
3,break dead shoot headquart french satir weekli...,Wed Jan 07 11:09:50 +0000 2015,15,105,AlArabiya_Eng,Sat Feb 28 08:31:32 +0000 2009,499741,31,100466,False,True,2,[],charliehebdo-all-rnr-threads,true
4,reuter peopl shot dead headquart french satir ...,Wed Jan 07 11:11:16 +0000 2015,32,412,SkyNewsBreak,Wed Nov 04 11:25:45 +0000 2009,1377384,6,27228,False,True,1,[],charliehebdo-all-rnr-threads,true
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6420,sydney sieg end polic storm lindt cafe hostag run,Mon Dec 15 15:55:59 +0000 2014,18,100,guardian,Thu Nov 05 23:49:19 +0000 2009,2997545,1081,102254,False,True,0,"[{'url': 'http://t.co/ek1EyHbJxA', 'indices': ...",sydneysiege-all-rnr-threads,false
6421,break news sydneysieg accord polic,Mon Dec 15 15:56:41 +0000 2014,315,489,FoxNews,Sat Mar 17 19:01:26 +0000 2007,4636658,382,188589,False,True,1,"[{'url': 'http://t.co/7p9QLG6mn4', 'indices': ...",sydneysiege-all-rnr-threads,false
6422,watch gunfir erupt hostag flee chocol shop syd...,Mon Dec 15 15:58:05 +0000 2014,91,181,NBCNews,Tue Mar 18 23:19:17 +0000 2008,1955488,2894,44356,False,True,1,"[{'url': 'http://t.co/Ju5h01y3R4', 'indices': ...",sydneysiege-all-rnr-threads,false
6423,author confirm sydneysieg polic storm build,Mon Dec 15 15:58:27 +0000 2014,35,117,YahooNewsUK,Fri Feb 13 15:44:54 +0000 2009,36137,926,14145,False,True,1,"[{'url': 'http://t.co/IrteddYrbH', 'indices': ...",sydneysiege-all-rnr-threads,false


In [14]:
# Export dataframe
%cd ..\..\datasets
df.to_csv('pheme.csv')
df2.to_csv('pheme2.csv')

c:\Users\willc\Documents\wills ensemble\datasets
